In [1]:
from env import config
import requests

In [2]:
MINDSDB_UN = config("MINDSDB_UN", default=None)
MINDSDB_PW = config("MINDSDB_PW", default=None)
assert MINDSDB_UN is not None
assert MINDSDB_PW is not None


MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [3]:
def get_mindsdb_session():
    session = requests.Session()
    session.post('https://cloud.mindsdb.com/cloud/login', json={
        'email': MINDSDB_UN,
        'password': MINDSDB_PW
    })
    return session

In [4]:
def mindsdb_query(session, sql_query):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    return session.post(url, json={"query": sql_query}, headers=headers)

In [29]:
def predict_query(session, 
                  flightDate="2022-04-21", 
                  startingAirport="LAX", 
                  isNonStop=1, 
                  destinationAirport="JFK",
                  raw_request=False
                 ):
    sql_query = f"""
    SELECT m.flightDate, m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence 
    FROM mindsdb.flight_price_predictor AS m
    JOIN ai_travel_agent.flight_prices AS t
    WHERE t.flightDate = "{flightDate}"
    AND t.startingAirport = "{startingAirport}"
    AND t.isNonStop = {isNonStop}
    AND t.destinationAirport = "{destinationAirport}";
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = query_response.json()
    columns = data.get("column_names")
    dataset = data.get("data")
    web_ready_data = [dict(zip(columns, row)) for row in dataset]
    return web_ready_data

In [36]:
session = get_mindsdb_session()
query_response = predict_query(session, startingAirport="SFO", destinationAirport="BOS", raw_request=False)
session.close()

In [37]:
data = query_response.json()
data.keys()

AttributeError: 'list' object has no attribute 'json'

In [38]:
print(query_response)

[{'flightDate': '2022-04-21 00:00:00.000000', 'segmentsAirlineName': 'Delta', 'isNonStop': '1', 'totalFare': 352.59628080569775, 'totalFare_confidence': 0.9991}, {'flightDate': '2022-04-21 00:00:00.000000', 'segmentsAirlineName': 'American Airlines', 'isNonStop': '1', 'totalFare': 404.39715167400135, 'totalFare_confidence': 0.9991}, {'flightDate': '2022-04-21 00:00:00.000000', 'segmentsAirlineName': 'JetBlue Airways', 'isNonStop': '1', 'totalFare': 354.99898339656704, 'totalFare_confidence': 0.9991}, {'flightDate': '2022-04-21 00:00:00.000000', 'segmentsAirlineName': 'United', 'isNonStop': '1', 'totalFare': 347.99920213019533, 'totalFare_confidence': 0.9991}, {'flightDate': '2022-04-21 00:00:00.000000', 'segmentsAirlineName': 'Alaska Airlines', 'isNonStop': '1', 'totalFare': 426.23859268457585, 'totalFare_confidence': 0.9991}]


In [39]:
columns = data.get("column_names")
dataset = data.get("data")
columns

['flightDate',
 'segmentsAirlineName',
 'isNonStop',
 'totalFare',
 'totalFare_confidence']

In [40]:
dataset

[['2022-04-21 00:00:00.000000', 'Delta', '1', 352.59628080569775, 0.9991],
 ['2022-04-21 00:00:00.000000',
  'American Airlines',
  '1',
  404.39715167400135,
  0.9991],
 ['2022-04-21 00:00:00.000000',
  'JetBlue Airways',
  '1',
  354.99898339656704,
  0.9991],
 ['2022-04-21 00:00:00.000000', 'United', '1', 347.99920213019533, 0.9991],
 ['2022-04-21 00:00:00.000000',
  'Alaska Airlines',
  '1',
  426.23859268457585,
  0.9991]]

In [42]:
query_response

[{'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Delta',
  'isNonStop': '1',
  'totalFare': 352.59628080569775,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'American Airlines',
  'isNonStop': '1',
  'totalFare': 404.39715167400135,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'JetBlue Airways',
  'isNonStop': '1',
  'totalFare': 354.99898339656704,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'United',
  'isNonStop': '1',
  'totalFare': 347.99920213019533,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Alaska Airlines',
  'isNonStop': '1',
  'totalFare': 426.23859268457585,
  'totalFare_confidence': 0.9991}]